In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from dataset import PalindromeDataset
from lstm import LSTM
from train import train, evaluate
from utils import AverageMeter, accuracy
from tqdm import tqdm

print(1)
input_length =  19
input_dim = 1  
num_classes = 10  
num_hidden = 128  
batch_size = 128  
learning_rate = 0.001 
max_epoch = 100 
max_norm = 10.0  
data_size = 100000  
portion_train = 0.8  

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 初始化数据集
dataset = PalindromeDataset(input_length, data_size)
# 划分训练集和验证集
train_size = int(portion_train * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# 创建训练集和验证集的数据加载器
train_dloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=batch_size)

# 初始化模型
model = LSTM(input_length, input_dim, num_hidden, num_classes)
model.to(device)

# 设置损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in tqdm(range(max_epoch)):
#     print(f'Epoch {epoch + 1}/{max_epoch}')
    # 训练模型一个epoch
    train_loss, train_acc = train(model, train_dloader, optimizer, criterion, device, {'max_norm': max_norm})
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
#     print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

    # 在验证集上评估训练好的模型
    val_loss, val_acc = evaluate(model, val_dloader, criterion, device, {})
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
#     print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    scheduler.step()

print(f'After {max_epoch} Epochs:')
print(f'Train Acc: {train_accuracies[-1] * 100:.4f}%, Validate Acc: {val_accuracies[-1] * 100:.4f}%')
print(f'Train Loss: {train_losses[-1]:.6f}, Validate Loss: {val_losses[-1]:.6f}')
    
print('Done training.')

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1
Using device: cuda


  2%|▏         | 2/100 [00:29<24:18, 14.88s/it]


KeyboardInterrupt: 

## Darw curve

In [ ]:
epochs = range(max_epoch)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs,train_losses, label='Train Loss')
plt.plot(epochs,val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(epochs,train_accuracies, label='Train Acc')
plt.plot(epochs,val_accuracies, label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()